<a href="https://colab.research.google.com/github/mredelis/CAP4453-Robot-Vision-Spr23/blob/main/RV_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 2#

##CAP 4453: Robot Vision##

Programming language for the assignment is Python and you will use PyTorch framework for deep learning.

Deliver the project as a colab note.

The colab note must include your code (properly running), and a short write-up about the results and your observations from each task. For each task, you should report the training/testing accuracy for the best model. Analyze the variation in training/testing loss as you train your network and discuss what you observe. Also, discuss the time required for training your network.


## Part 1. Neural Networks ##

You goal in this assignment is to train neural networks for digit classiﬁcation. You will use MNIST dataset which has around 70K images of handwritten digits. You will be provided the template code for this assignment, and you must make some changes to the network and analyze the results after these changes.

Your tasks:
  1. [15%] Simple neural network: In this task, your goal is to design a neural network with 3 layers (input, hidden,and output) and in each layer you should use less than 20 neurons. There should be NO activation functions in your network. There are 10 classes in MNIST dataset corresponding to each digit, so this will be a 10-way classiﬁcation network.

  2. [15%] Activation function: In this task, you will add activation function to your network. Use ReLU activation in all your layers from the previous task.
  
  3. [20%] Deep network: In this task, your goal is to increase the size of the network. Your new network should have more than 4 layers and each layer should have more than 200 neurons. Use ReLU activation in all the layers. Note that the last layer will still have only 10 neurons as this is a 10-way classiﬁcation network.


### Load and normalize MNIST ###

In [ ]:
# Load libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision # dataloader for common datasets including CIFAR10
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Relevant for variables for Part 1. These are the same for tasks 1, 2, and 3
# batch_size_part1 = 10
# learning_rate_part1 = 0.1
# num_epochs_part1 = 20
# batch_size_part1 = 64
# learning_rate_part1 = 1e-3
# num_epochs_part1 = 20
# batch_size_part1 = 10
# learning_rate_part1 = 1e-3
# num_epochs_part1 = 20
batch_size_part1 = 100
learning_rate_part1 = 0.1
num_epochs_part1 = 10

# Create transformations to apply to each data sample 
# Can specify variations such as image flip, color flip, random crop, ...
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

dataset1 = torchvision.datasets.MNIST('./data/', 
                                      train=True, 
                                      download=True,
                                      transform=transform)

dataset2 = torchvision.datasets.MNIST('./data/', 
                                      train=False,
                                      transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, 
                                           batch_size=batch_size_part1, 
                                           shuffle=True, 
                                           num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset2, 
                                          batch_size=batch_size_part1, 
                                          shuffle=False, 
                                          num_workers=2)

### Define Convolutional Neural Network for Tasks 1, 2, 3 ###

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()
        
        # Define various layers
        self.fc1 = nn.Linear(in_features=28*28, out_features=100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 10)

        # More than 4 layers and each layer should have more than 200 neurons
        self.fc_layer1 = nn.Linear(in_features=28*28, out_features=700)
        self.fc_layer2 = nn.Linear(700, 500)
        self.fc_layer3 = nn.Linear(500, 400)
        self.fc_layer4 = nn.Linear(400, 10)
        
        # This will select the forward pass function based on mode for the ConvNet.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        else: 
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)
             
    # task 1
    def model_1(self, X):
        # Three fully connected layers without activation
        X = torch.flatten(X, start_dim=1)
        X = self.fc1(X)  
        X = self.fc2(X)  
        X = self.fc3(X)    
                        
        return X
        
    # task 2
    def model_2(self, X):
        # Train with activation (use model 1 from task 1)
        X = torch.flatten(X, start_dim=1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        
        return X

    # task 3
    def model_3(self, X):
        # Change number of fully connected layers and number of neurons from model 2 in task 2       
        X = torch.flatten(X, start_dim=1)
        X = F.relu(self.fc_layer1(X))
        X = F.relu(self.fc_layer2(X))
        X = F.relu(self.fc_layer3(X))
        X = F.relu(self.fc_layer4(X))

        return X

## Train and test functions ##

In [ ]:
def train_part1(model, train_loader, optimizer, criterion,epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. 
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        data, target = batch_sample

        # zero the parameter gradients     
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data) 

        # Compute loss based on criterion
        loss = criterion(output, target)
        # Computes gradient based on final loss
        loss.backward()     
        # Store loss
        losses.append(loss.item())
        # print(f"Losses in train: {losses}\n")
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        # Count correct predictions overall 
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    
    return train_loss, train_acc

In [ ]:
def test_part1(model, test_loader):
    '''
    Tests the model.
    model: The model to train. 
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()   
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
      for batch_idx, sample in enumerate(test_loader):
          data, target = sample      

          # Predict for data by doing forward pass
          output = model(data)
          
          # Compute loss based on same criterion as training
          loss = F.cross_entropy(output, target, reduction='mean')
          
          # Append loss to overall test loss
          losses.append(loss.item())

          pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability

          # Count correct predictions overall 
          correct += pred.eq(target.view_as(pred)).sum().item()
          

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy

### Instantiate network, define a loss function and optimizer ###

In [ ]:
import torch.optim as optim
import time
import datetime 
from datetime import timedelta

for i in range(1, 4): # Models 1, 2, 3

  print(f"Task {i}: Model {i}\n")
  t = time.process_time() # to keep track of time

  # Instantiate network
  model = ConvNet(i)

  # Initialize the criterion for loss computation 
  criterion = torch.nn.CrossEntropyLoss() 

  optimizer = optim.SGD(model.parameters(), lr=learning_rate_part1, weight_decay=1e-7)

  best_accuracy = 0.0
  for epoch in range(1, num_epochs_part1+1):
    print("\nEpoch: ", epoch)
    train_loss, train_accuracy = train_part1(model, train_loader, optimizer, criterion , epoch, batch_size_part1)
    test_loss, test_accuracy = test_part1(model, test_loader)
    
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy

  print()     
  print("Accuracy is {:2.2f}".format(best_accuracy))

  print(f"Training and evaluation finished for Task {i}: Model {i}\n")

  elapsed_time = time.process_time() - t
  print(f"Elapsed time: {str(datetime.timedelta(seconds=elapsed_time))}\n")

Task 1: Model 1


Epoch:  1
Train set: Average loss: 0.3950, Accuracy: 53218/60000 (89%)
Test set: Average loss: 0.3077, Accuracy: 9117/10000 (91%)

Epoch:  2
Train set: Average loss: 0.3128, Accuracy: 54644/60000 (91%)
Test set: Average loss: 0.2991, Accuracy: 9159/10000 (92%)

Epoch:  3
Train set: Average loss: 0.3032, Accuracy: 54823/60000 (91%)
Test set: Average loss: 0.2955, Accuracy: 9166/10000 (92%)

Epoch:  4
Train set: Average loss: 0.2959, Accuracy: 54990/60000 (92%)
Test set: Average loss: 0.2882, Accuracy: 9183/10000 (92%)

Epoch:  5
Train set: Average loss: 0.2902, Accuracy: 55023/60000 (92%)
Test set: Average loss: 0.2872, Accuracy: 9181/10000 (92%)

Epoch:  6
Train set: Average loss: 0.2860, Accuracy: 55160/60000 (92%)
Test set: Average loss: 0.3008, Accuracy: 9138/10000 (91%)

Epoch:  7
Train set: Average loss: 0.2845, Accuracy: 55153/60000 (92%)
Test set: Average loss: 0.2945, Accuracy: 9210/10000 (92%)

Epoch:  8
Train set: Average loss: 0.2819, Accuracy: 55236/60000 

## Discuss results ##

|                        | Task 1      |Task 2    |Task 3     |
| ---------------------  | ----------- |----------|---------- |
| Training Accuracy      | 92%         |99%       |100%       |
| Test Accuracy          | 92%         |98%       |98%        |
| Training time in Colab | 37 sec      |38 sec    |~2 1/2 min |

<br> 

The neural network for Task 1 does not have an activation function and every neuron will only perform a linear transformation using the weights and biases. Even though, the accuracy yield is high because the MNIST dataset is a very easy-to-learn dataset with few complex patterns from the data. 

The accuracy is improved in Task 2 when the ReLu activation function is added to all the layers from Task 1. The activation function introduces the non-linearity to the network helping the model to learn important information while suppressing the irrelevant data points by normalizing the output of any neuron between 1 and 0.  

Accuracy is slightly improved in Task 3 given the network has more neurons. But the main difference from Task 2 is the increase in the time it takes to train the network.   

## Part 2. Convolutional Neural Network ##

Your goal in this assignment is to train convolutional neural networks for image classiﬁcation. You will use CIFAR-10 dataset, which has 60K color images (each has size 32x32 pixels) from 10 classes. You will be provided the template code for this assignment, and you have to make some changes to the network and analyze the results after these changes. For each of these tasks, use learning rate of 0.1 and batch size of 100 and train them for 10 epochs each.
 `
Your tasks:

  4. [20%] Simple CNN: In this task, your goal is to design a convolutional neural network with 2 convolutional layers (Conv2d) layers and 2 pooling layers, followed by 2 fully connected layers. Both Conv2d layers should have 10 ﬁlters (output channels). The second Conv2d layer’s input channels should match ﬁrst Conv2d layer’s output channels. Use a kernel size of 3 for all convolutional layers. Apply ReLU activation to each Conv2d. Each Conv2d layer should be followed by max_pool2d layer with kernel size of 2. The output features from convolution after ﬂattening will be 360, so set the input features in fully connected layer accordingly (You can use fc1_model1) for this. There are 10 classes in CIFAR-10 dataset, so this will be a 10-way
classiﬁcation network. You can use model_0 from the template and modify that to ﬁt this task.

  5. [15%] Increase ﬁlters: In this task, you will increase the ﬁlters in each Conv2d layer in your network. Learn 20 kernels for the ﬁrst Conv2d layer(set output channels to 20). For the second Conv2d layer learn 40 kernels (set the output channels to 40). Match input channels for second Conv2d layer with output channels of ﬁrst Conv2d layer. Since this will change output feature size after second Conv2d layer, use fc1_model2 with 1440 input features for this task.

  6. [15%] Large CNN: In this task, your goal is to increase the size of the network. Take the network from previous task and add one more Conv2d layer with 40 ﬁlters (set both input and output channels to 40). Do not add a max pooling layer after this third convolution layer. Use fc1_model3 with 640 input features for this task.


### Load and normalize CIFAR-10 ###

In [ ]:
# Load libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision # dataloader for common datasets including CIFAR10
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Relevant for variables for Part 2. These are the same for tasks 4, 5, and 6
batch_size = 100
learning_rate = 0.1
num_epochs = 10

num_classes = 10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Output of torchvision datasets are PILImage images of range [0, 1]. 
# Transform them to Tensors of normalized range [-1, 1].
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Define datasets
trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True, 
                                        transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, 
                                          batch_size=batch_size,
                                          shuffle=True, 
                                          num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', 
                                       train=False,
                                       download=True, 
                                       transform=transform)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=batch_size,
                                         shuffle=False, 
                                         num_workers=2)

# functions to show an image
# def imshow(img):
#   img = img / 2 + 0.5     # unnormalize
#   npimg = img.numpy()
#   plt.imshow(np.transpose(npimg, (1, 2, 0)))
#   plt.show()

# get some random training images
# dataiter = iter(trainloader)
# images, labels = next(dataiter)

# for i in range(2):  # show just the frogs
#   imshow(torchvision.utils.make_grid(images[i]))

Files already downloaded and verified
Files already downloaded and verified


### Define Convolutional Neural Network for Tasks 4, 5, 6 ###

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, mode):
        super(ConvNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3)

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=20, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=40, out_channels=40, kernel_size=3)

        self.fc1_model1 = nn.Linear(360, 100)  # This is first fully connected layer for step 1.
        self.fc1_model2 = nn.Linear(1440, 100) # This is first fully connected layer for step 2.
        self.fc1_model3 = nn.Linear(640, 100)  # This is first fully connected layer for step 3

        self.fc2 = nn.Linear(100, 10)       # This is 2nd fully connected layer for all models.
        
        # This will select the forward pass function based on mode for the ConvNet.
        # Based on the question, there are 3 modes available for tasks 4, 5 and 6.
        # During creation of each ConvNet model, you will assign one of the valid mode.
        # This will fix the forward function (and the network graph) for the entire training/testing
        if mode == 1:
            self.forward = self.model_1
        elif mode == 2:
            self.forward = self.model_2
        elif mode == 3:
            self.forward = self.model_3
        else: 
            print("Invalid mode ", mode, "selected. Select between 1-3")
            exit(0)

    # Model for Task 4 Simple CNN
    def model_1(self, x): 
      x = self.conv_layer1(x) 
      x = F.relu(x)  
      x = F.max_pool2d(x, kernel_size=2) 
      x = self.conv_layer2(x)  
      x = F.relu(x)  
      x = F.max_pool2d(x, kernel_size=2)   
      x = torch.flatten(x, start_dim=1) # flatten all dimensions except batch 
      x = F.relu(self.fc1_model1(x))
      x = self.fc2(x)
        
      return x

    # Model for Task 5 Increase filters
    def model_2(self, x): 
      x = F.max_pool2d(F.relu(self.conv1(x) ), kernel_size=2) 
      x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2)   
      x = torch.flatten(x, start_dim=1) # flatten all dimensions except batch 
      x = F.relu(self.fc1_model2(x))
      x = self.fc2(x)
        
      return x

    # Model for Task 6 Increase filters
    def model_3(self, x): 
      x = F.max_pool2d(F.relu(self.conv1(x) ), kernel_size=2) 
      x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2) 
      x = F.relu(self.conv3(x))   
      x = torch.flatten(x, start_dim=1) # flatten all dimensions except batch 
      x = F.relu(self.fc1_model3(x))
      x = self.fc2(x)
        
      return x

## Train and test functions ##

In [ ]:
def train(model, train_loader, optimizer, criterion, epoch, batch_size):
    '''
    Trains the model for an epoch and optimizes it.
    model: The model to train. 
    train_loader: dataloader for training samples.
    optimizer: optimizer to use for model parameter updates.
    criterion: used to compute loss for prediction and target 
    epoch: Current epoch to train for.
    batch_size: Batch size to be used.
    '''
    
    # Set model to train mode before each epoch
    model.train()
    
    # Empty list to store losses 
    losses = []
    correct = 0
    
    # Iterate over entire training samples (1 epoch)
    for batch_idx, batch_sample in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        data, target = batch_sample

        # zero the parameter gradients     
        # Reset optimizer gradients. Avoids grad accumulation (accumulation used in RNN).
        optimizer.zero_grad()
        
        # Do forward pass for current set of data
        output = model(data)     
        # Compute loss based on criterion
        loss = criterion(output, target)       
        # Computes gradient based on final loss
        loss.backward()
        
        # Store loss
        losses.append(loss.item())
        
        # Optimize model parameters based on learning rate and gradient 
        optimizer.step()
        
        # Get predicted index by selecting maximum log-probability
        pred = output.argmax(dim=1, keepdim=True)
        
        # Count correct predictions overall 
        correct += pred.eq(target.view_as(pred)).sum().item()     

    train_loss = float(np.mean(losses))
    train_acc = correct / ((batch_idx+1) * batch_size)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        float(np.mean(losses)), correct, (batch_idx+1) * batch_size,
        100. * correct / ((batch_idx+1) * batch_size)))
    
    return train_loss, train_acc

In [ ]:
def test(model, test_loader):
    '''
    Tests the model.
    model: The model to train. 
    test_loader: dataloader for test samples.
    '''
    
    # Set model to eval mode to notify all layers.
    model.eval()
    
    losses = []
    correct = 0
    
    # Set torch.no_grad() to disable gradient computation and backpropagation
    with torch.no_grad():
      for batch_idx, sample in enumerate(test_loader):
          data, target = sample      

          # Predict for data by doing forward pass
          output = model(data)
          
          # Compute loss based on same criterion as training
          loss = F.cross_entropy(output, target, reduction='mean')
          
          # Append loss to overall test loss
          losses.append(loss.item())
          
          # Get predicted index by selecting maximum log-probability
          pred = output.argmax(dim=1, keepdim=True)
          
          # Count correct predictions overall 
          correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss = float(np.mean(losses))
    accuracy = 100. * correct / len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    return test_loss, accuracy

### Instantiate network, define a loss function and optimizer ###

In [ ]:
import torch.optim as optim
import time
import datetime 
from datetime import timedelta

for i in range(1, 4): # Models 1, 2, 3

  print(f"Task {i+3}: Model {i}\n")
  t = time.process_time() # to keep track of time

  # Instantiate network
  model = ConvNet(i)

  # Initialize the criterion for loss computation 
  criterion = nn.CrossEntropyLoss(reduction='mean')

  # Initialize optimizer type 
  optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-7)

  best_accuracy = 0.0
  for epoch in range(1, num_epochs+1):
    print("\nEpoch: ", epoch)
    train_loss, train_accuracy = train(model, trainloader, optimizer, criterion, epoch, batch_size)
    test_loss, test_accuracy = test(model, testloader)
    
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy

  print()       
  print("Accuracy is {:2.2f}".format(best_accuracy))

  print(f"Training and evaluation finished for Task {i+3}: Model {i}\n")

  elapsed_time = time.process_time() - t
  print(f"Elapsed time: {str(datetime.timedelta(seconds=elapsed_time))}\n")

Task 4: Model 1


Epoch:  1
Train set: Average loss: 1.8305, Accuracy: 16899/50000 (34%)
Test set: Average loss: 1.6324, Accuracy: 4003/10000 (40%)

Epoch:  2
Train set: Average loss: 1.4796, Accuracy: 23277/50000 (47%)
Test set: Average loss: 1.3479, Accuracy: 5147/10000 (51%)

Epoch:  3
Train set: Average loss: 1.3274, Accuracy: 26231/50000 (52%)
Test set: Average loss: 1.2766, Accuracy: 5430/10000 (54%)

Epoch:  4
Train set: Average loss: 1.2284, Accuracy: 28236/50000 (56%)
Test set: Average loss: 1.2018, Accuracy: 5748/10000 (57%)

Epoch:  5
Train set: Average loss: 1.1472, Accuracy: 29783/50000 (60%)
Test set: Average loss: 1.1816, Accuracy: 5859/10000 (59%)

Epoch:  6
Train set: Average loss: 1.0817, Accuracy: 31086/50000 (62%)
Test set: Average loss: 1.1924, Accuracy: 5825/10000 (58%)

Epoch:  7
Train set: Average loss: 1.0275, Accuracy: 32045/50000 (64%)
Test set: Average loss: 1.1006, Accuracy: 6136/10000 (61%)

Epoch:  8
Train set: Average loss: 0.9850, Accuracy: 32770/50000 

## Discuss results ##

|                        | Task 4      |Task 5      |Task 6    |
| ---------------------  | ----------- |----------- |----------|
| Training Accuracy      | 68%         |80%         |74%       |
| Test Accuracy          | 62%         |69%         |70%       |
| Training time in Colab | ~3 1/2 min  |~6 1/2 min  |~7 min    |

<br>

Starting from the simple CNN (task 4), task 5 increases the number of filters from 10 to 20 in the first Conv2d and from 10 to 40 in the second Conv2d improving the accuracy of the CNN. The number of filters is increased to increase the depth of the feature space (feature map) which helps in learning more levels of global abstract structures, and therefore, a more powerful model. 

Task 6 adds an additional convolutional layer for a total of 3 convolutional layers, but it does not have a max pooling operation after the third convolutional layer. As shown in the table above, it takes more time due to the additional convolution operation (element-wise matrix multiplication and addition) of the 3rd layer.